In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"
# import tensorflow as tf
import numpy as np
import cv2
import shutil
import seaborn as sns
import random
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
%matplotlib inline

In [ ]:
desired_size = 128

In [ ]:
def resize_and_pad(im, desired_shape = 128):
    old_size = im.shape[:2] # old_size is in (height, width) format
    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])

    # new_size should be in (width, height) format
    im = cv2.resize(im, (new_size[1], new_size[0]))

    delta_w = desired_size - new_size[1]
    delta_h = desired_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)

    color = [0, 0, 0]
    new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,
        value=color)
    return new_im
def histequal(img):
    img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)

    # equalize the histogram of the Y channel
    img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])

    # convert the YUV image back to RGB format
    img_output = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)
    
    return img_output
def histogram(img):
    hist,bins = np.histogram(img.flatten(),256,[0,256])
    cdf = hist.cumsum()
    cdf_normalized = cdf * hist.max()/ cdf.max()
    # plt.plot(cdf_normalized, color = 'g')
    plt.hist(img.flatten(),256,[0,256], color = 'b')
    plt.xlim([0,256])
    plt.legend(('cdf','histogram'), loc = 'upper left')
    plt.show()

In [ ]:
from sklearn import datasets, svm, metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA

train_data = np.load("Data/Train_data.npz")
train_data = train_data.f.arr_0
train_labels = np.load("Data/Train_labels.npz")
train_labels = train_labels.f.arr_0
val_data = np.load("Data/Validation_data.npz")
val_data = val_data.f.arr_0
val_labels = np.load("Data/Validation_labels.npz")
val_labels = val_labels.f.arr_0
test_data = np.load("Data/Test_data.npz")
test_data = test_data.f.arr_0
test_labels = np.load("Data/Test_labels.npz")
test_labels = test_labels.f.arr_0

val_data = val_data.reshape((val_data.shape[0], -1))
test_data = test_data.reshape((test_data.shape[0], -1))
m = train_data.shape[0]

train_data = train_data.reshape((m, -1))

print("Train Data : " + str(train_data.shape))
print("Train Labels : " + str(train_labels.shape))

print("Validation Data : " + str(val_data.shape))
print("Validation Labels : " + str(val_labels.shape))

print("Test Data : " + str(test_data.shape))
print("Test Labels : " + str(test_labels.shape))

In [ ]:
train_data = np.concatenate((train_data, val_data))
train_labels = np.concatenate((train_labels, val_labels))
print("Train Data : " + str(train_data.shape))
print("Train Labels : " + str(train_labels.shape))

In [ ]:
# from urllib.request import urlopen

def get_img(image_url):
    # image_url = "https://3.imimg.com/data3/QD/HN/MY-678214/helmet-250x250.jpg"
    # image_url = "https://images-na.ssl-images-amazon.com/images/I/51Q0Ph9JmdL.jpg"
    f = open('test.jpg','wb')
    f.write(urlopen(image_url).read())
    f.close()

    img = cv2.imread("test.jpg")
    imshow(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = resize_and_pad(img)
    img = cv2.equalizeHist(img)
    img = img / 255
    return img

# SVM

In [ ]:
pca = PCA(n_components=300)

principalComponents = pca.fit_transform(train_data)
print(principalComponents.shape)
classifier = svm.SVC(kernel = 'rbf')
classifier.fit(principalComponents, train_labels)

In [ ]:
expected = test_labels
pc_test = pca.transform(test_data)
print(pc_test.shape)
predicted = classifier.predict(pc_test)

print("Classification report for classifier %s:\n%s\n"
      % (classifier, metrics.classification_report(expected, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

print("Accuracy : " + str(accuracy_score(expected, predicted)))

# Random Forest

In [72]:
clf = RandomForestClassifier(n_estimators = 144)
clf = clf.fit(train_data, train_labels)

In [75]:
expected = test_labels
predicted = clf.predict(test_data)

print("Classification report for classifier %s:\n%s\n"
      % (clf, metrics.classification_report(expected, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

print("Accuracy : " + str(accuracy_score(expected, predicted)))

Classification report for classifier RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=144, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False):
             precision    recall  f1-score   support

          0       1.00      0.75      0.86        12
          1       1.00      1.00      1.00        18
          2       0.95      0.95      0.95        40
          3       1.00      0.94      0.97        18
          4       0.83      0.88      0.85        33
          5       0.83      0.99      0.90        69
          6       0.97      1.00      0.98        28
          7       1.00      1.00      1.00        15
          8       0.96      0.72      0.83        36
         

In [69]:
import pickle
pickle.dump(clf, open("random_forest_model.sav", 'wb'))

In [74]:
clf = pickle.load(open("random_forest_model.sav", 'rb'))

# Adaboost

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import AdaBoostClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import mean_squared_error

param_grid = {"base_estimator__splitter" :   ["best", "random"],
             "n_estimators": [ 5, 10, 50, 100, 300],
             "learning_rate": [0.1, 1]
            }

dectree = DecisionTreeClassifier(random_state = 11, max_features = "auto", max_depth = None)

adb = AdaBoostClassifier(base_estimator = dectree)

# run grid search
gridsearchADB = GridSearchCV(adb, param_grid=param_grid, cv=5, n_jobs=-1)
gridsearchADB.fit(train_data, train_labels)

In [ ]:
expected = test_labels
predicted = gridsearchADB.predict(test_data)

print("Classification report for classifier %s:\n%s\n"
      % (gridsearchADB, metrics.classification_report(expected, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

print("Accuracy : " + str(accuracy_score(expected, predicted)))